In [12]:
NEUROGLANCER_ROOT = '/media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/'

In [13]:
import socket
import neuroglancer
import subprocess
import numpy as np
from neuroglancer.server import global_server_args

class Neuroglancer_Viewer:
    def __init__(self, stack='MD585'):
        self.stack = stack
        self.local_volume_fp_root = './'
        
        neuroglancer.set_server_bind_address('0.0.0.0')
        global_server_args['bind_port'] = 80
        
        # Create viewer
        self.viewer = neuroglancer.Viewer()
        
        # Get the IP address
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip_name = s.getsockname()[0]
        s.close()

        # Removes the following symbols: ', ", [, ]
        self.url = str( 'http://' + ip_name + ':' + self.viewer.get_viewer_url().split(':')[2] )##Remote URL
        
    def set_local_volume_fp(self, fp):
        if fp[len(fp)-1]!='/':
            fp = fp+'/'
        self.local_volume_fp_root = fp
        
    def download_volumes(self):
        s3_root_fp = 's3://test-bucket-sid/alex_neuroglancer_volumes/'+self.stack+'/human_annotations_5um/'
        local_volume_fp = self.local_volume_fp_root+self.stack+'/human_annotations_5um/'
        
        command_list = ['aws','s3','cp','--recursive', s3_root_fp, local_volume_fp]
        subprocess.call( command_list )
    
    def add_stack(self):
        if self.stack == 'MD585':
            with self.viewer.txn() as s:
                s.layers[ self.stack+'_image'] = neuroglancer.ImageLayer(\
                    source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/MD585_fullres')
        elif self.stack == 'MD589':
            with self.viewer.txn() as s:
                s.layers[ self.stack+'_image'] = neuroglancer.ImageLayer(\
                    source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/MD589_fullres')
        elif self.stack == 'MD594':
            with self.viewer.txn() as s:
                s.layers[ self.stack+'_image'] = neuroglancer.ImageLayer(\
                    source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/MD594_fullres')
            
    def add_volume(self, colored=True):
        volume_fp = self.local_volume_fp_root+self.stack+'/human_annotation/solid_volume_5um/'
        if colored:
            volume_fn = 'volume_colored.npy'
            color_segments=[]
            for i in range(1,50):
                color_segments.append(i)
        else:
            volume_fn = 'volume.npy'
        
        xy_ng_resolution_um = 5
        volume_data = np.load( volume_fp+volume_fn )
        
        with self.viewer.txn() as s:
            s.layers[ self.stack+"_Annotations" ] = neuroglancer.SegmentationLayer(
                source = neuroglancer.LocalVolume(
                    data=volume_data, # Z,Y,X
                    voxel_size=[ xy_ng_resolution_um*1000, xy_ng_resolution_um*1000, 20000], # X Y Z
                    voxel_offset = [ 0, 0, 0] # X Y Z
                ),
                segments = color_segments
            )
            
    def reset_orientation(self):
        with self.viewer.txn() as s:
            # Resets X/Y/Z plane orientation
            s.navigation.pose.orientation = [0,0,0,1]
            # Zooms out 
            s.navigation.zoomFactor = 5000 # 5000 If xy, 10000 If 4panel
            # Resets 3D Viewer Orientation
            s.perspectiveOrientation = [0,0,0,1]
            # Zooms out
            s.perspectiveZoom = 75000

            s.layout = 'xy' # '3d'/'4panel'/'xy'
            
    def stop(self):
        neuroglancer.stop()

In [14]:
new_viewer = Neuroglancer_Viewer(stack='MD594') # 8 off
new_viewer.add_stack()
new_viewer.reset_orientation()

print new_viewer.url

http://132.239.73.85:38039/v/9fa0f8ef67cf0fb604e27e71154fe204cb504ded/


In [16]:
new_viewer = Neuroglancer_Viewer(stack='MD589') # 8 off
new_viewer.add_stack()
new_viewer.reset_orientation()

print new_viewer.url

http://132.239.73.85:39189/v/d9fff74df8704d687de2599626464c2eeab28d13/


In [18]:
new_viewer = Neuroglancer_Viewer(stack='MD585')
new_viewer.add_stack()

new_viewer.set_local_volume_fp( fp=NEUROGLANCER_ROOT )
new_viewer.add_volume()

new_viewer.reset_orientation()

print new_viewer.url

http://132.239.73.85:35671/v/22c415e5d52c576d85fdcc01af7376bfebc9929d/


In [17]:
new_viewer.stop()
del new_viewer

In [57]:
import subprocess
s3_root_fp = 's3://test-bucket-sid/alex_neuroglancer_volumes/'+'MD585'+'/human_annotations_5um'
        
command_list = ['aws','s3','cp','--recursive',s3_root_fp, '/home/alexn/Desktop/test_ng/']
subprocess.call( command_list )

1

In [41]:
! rm -r /home/alexn/test_ng